# Pipeline para Ingeniería de Características para Inferencia en Transacciones de Tarjetas de Crédito

Este notebook implementa un pipeline de **ingeniería de características** diseñado para preparar un conjunto de datos de transacciones de tarjetas de crédito exclusivamente para tareas de **inferencia**. Está optimizado para ejecutarse en **Databricks Community Edition**, maximizando las capacidades de procesamiento de datos dentro de las limitaciones de la plataforma.

---

## Propósito General del Notebook

1. **Cargar datos crudos:** Importa datos desde un archivo CSV con información de transacciones de tarjetas de crédito.
2. **Ingeniería de características:** Realiza transformaciones clave para preparar los datos:
   - **Velocidad de transacciones:** Número de transacciones realizadas por cliente en una ventana temporal definida.
   - **Velocidad de montos:** Monto total gastado por cliente en la misma ventana temporal.
   - **Perfilado de categoría de comerciante:** Análisis de patrones de gasto por categoría comercial.
   - **Patrones temporales:** Extracción de características como la hora del día y el día de la semana.
3. **Preparación para inferencia:** Genera un conjunto de datos enriquecido listo para ser usado por modelos previamente entrenados.
4. **Almacenamiento en formato Delta:** Guarda los datos procesados en formato **Delta**, optimizado para consultas y análisis.

---

## Rutas Necesarias para la Configuración

### **Dependencias**
- Todas las dependencias necesarias están instaladas en la primera celda del notebook.

### **Archivos de Entrada**
- Los datos ya están en CSV por la ejecución del notebook anterior (`credit_card_transactions_dirt_to_inf.csv`).
- Ubicación en Databricks: `/FileStore/tables/`.

### **Archivos de Salida**
- **Datos procesados en formato Delta:** Los datos procesados se encontraran en `/FileStore/tables/output_delta_table_datapipe_feature_eng_to_inf` una vez finalizada la ejecución del notebook 

**Nota:** La escases de doc, unit tests y experimentos en este notebook es porque es igual al notebook anterior solo que cambiando el archivo de salida y entrada. 


In [0]:
!pip install imbalanced-learn
!pip install mlflow

  Using cached imbalanced_learn-0.12.4-py3-none-any.whl (258 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-5ca7bea9-b248-4e06-82d0-ce91bb64ef06/bin/python -m pip install --upgrade pip' command.
  Using cached mlflow-2.19.0-py3-none-any.whl (27.4 MB)
  Using cached SQLAlchemy-2.0.37-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl (114 kB)
  Using cached docker-7.1.0-py3-none-any.whl (147 kB)
  Using cached flask-3.1.0-py3-none-any.whl (102 kB)
  Using cached mlflow_skinny-2.19.0-py3-none-any.whl (5.9 MB)
  Using cached gunicorn-23.0.0-py3-none-any.whl (85 kB)
  Using cached Markdown-3.7-py3-none-any.whl (106 kB)
  Using cached alembic-1.14.1-py3-none-any.whl (233 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl (44 kB)
  Using cached opentelemetry_api-1.29.0-py3-none-any.whl (64 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl (118 kB)
  Using cached cloudpickle-3.1

# Data Pipeline and Feature Engineering

* ### Load and Process the Provided Credit Card Transaction Dataset 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, unix_timestamp, hour, dayofweek
from pyspark.sql.window import Window
import logging

def setup_logging():
    """Setup logging configuration."""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
    )
    return logging.getLogger(__name__)

logger = setup_logging()

def create_spark_session(app_name="CreditCardFraudDetection"):
    """Create and return a Spark session."""
    try:
        spark = SparkSession.builder \
            .appName(app_name) \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .getOrCreate()
        logger.info("Spark session created successfully.")
        return spark
    except Exception as e:
        logger.error("Failed to create Spark session: %s", str(e))
        raise

def load_data(spark, file_path):
    """Load data from the provided file path."""
    try:
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load(file_path)
        logger.info("Data loaded successfully from %s.", file_path)
        return df
    except Exception as e:
        logger.error("Failed to load data: %s", str(e))
        raise

def preprocess_data(df):
    """Initial preprocessing: Cast columns to appropriate data types."""
    try:
        df = df.withColumn("amount", col("amount").cast("double"))
        df = df.withColumn("timestamp", col("timestamp").cast("timestamp"))
        logger.info("Data preprocessing completed.")
        return df
    except Exception as e:
        logger.error("Error during preprocessing: %s", str(e))
        raise


### Transaction Velocity

In [0]:
def transaction_velocity(df):
    """Calculate transaction velocity (number of transactions per time window)."""
    try:
        df = df.withColumn("timestamp_seconds", unix_timestamp(col("timestamp")))
        time_window = Window.partitionBy("customer_id").orderBy("timestamp_seconds").rangeBetween(-604800, 0)
        df = df.withColumn("transaction_velocity", count("transaction_id").over(time_window))
        logger.info("Transaction velocity calculated.")
        return df
    except Exception as e:
        logger.error("Error calculating transaction velocity: %s", str(e))
        raise

### Amount Velocity

In [0]:
def amount_velocity(df):
    """Calculate amount velocity (total amount per time window)."""
    try:
        time_window = Window.partitionBy("customer_id").orderBy("timestamp_seconds").rangeBetween(-604800, 0)
        df = df.withColumn("amount_velocity", sum("amount").over(time_window))
        logger.info("Amount velocity calculated.")
        return df
    except Exception as e:
        logger.error("Error calculating amount velocity: %s", str(e))
        raise


### Merchant Category Profiling

In [0]:
def merchant_category_profiling(df):
    """Calculate merchant category profiling for each customer."""
    try:
        category_window = Window.partitionBy("customer_id", "merchant_category")
        df = df.withColumn("merchant_category_count", count("transaction_id").over(category_window))
        logger.info("Merchant category profiling completed.")
        return df
    except Exception as e:
        logger.error("Error during merchant category profiling: %s", str(e))
        raise

### Time-Based Patterns 

In [0]:
def time_based_patterns(df):
    """Extract time-based patterns like hour of day and day of week."""
    try:
        df = df.withColumn("hour_of_day", hour(col("timestamp")))
        df = df.withColumn("day_of_week", dayofweek(col("timestamp")))
        logger.info("Time-based patterns extracted.")
        return df
    except Exception as e:
        logger.error("Error extracting time-based patterns: %s", str(e))
        raise

### Implement Feature Engineering Using Spark SQL and Window Functions 

In [0]:
def full_feature_engineering(df):
    """Run all feature engineering steps sequentially."""
    try:
        df = transaction_velocity(df)
        df = amount_velocity(df)
        df = merchant_category_profiling(df)
        df = time_based_patterns(df)
        logger.info("All feature engineering steps completed.")
        return df
    except Exception as e:
        logger.error("Error during full feature engineering: %s", str(e))
        raise

### Store Engineered Features in Delta Format

In [0]:
def save_to_delta(df, output_path):
    """Save processed data to Delta format."""
    try:
        if not output_path:
            raise ValueError("Output path cannot be empty.")

        df.write.format("delta").mode("overwrite").save(output_path)
        logger.info("Data saved to Delta format at %s.", output_path)
    except Exception as e:
        logger.error("Failed to save data to Delta: %s", str(e))
        raise

### Production-Ready Code
* Main Function for Pipeline Execution

In [0]:
if __name__ == "__main__":
    try:
        # Setup
        logger.info("Starting the data pipeline.")
        spark = create_spark_session()

        # Load data
        file_path = "/FileStore/tables/credit_card_transactions_dirt_to_inf.csv"  # Replace with your file path
        df = load_data(spark, file_path)

        # Preprocess data
        df = preprocess_data(df)

        # Perform feature engineering
        df = full_feature_engineering(df)

        # Save the processed data
        output_path = "/FileStore/tables/output_delta_table_datapipe_feature_eng_to_inf"
        save_to_delta(df, output_path)

        logger.info("Data pipeline completed successfully.")
    except Exception as e:
        logger.error("Pipeline execution failed: %s", str(e))
        raise